In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


In [9]:
import pandas as pd
sample_submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
test = pd.read_csv("../input/nlp-getting-started/test.csv")
train = pd.read_csv("../input/nlp-getting-started/train.csv")

In [10]:
train.columns.values, test.columns.values

(array(['id', 'keyword', 'location', 'text', 'target'], dtype=object),
 array(['id', 'keyword', 'location', 'text'], dtype=object))

In [11]:
train[train['location'].notnull()]

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0


In [12]:
train['text'].str.extractall(r'(\#)')[0]

      match
0     0        #
3     0        #
4     0        #
      1        #
5     0        #
              ..
7599  1        #
7601  0        #
      1        #
7604  0        #
7607  0        #
Name: 0, Length: 3403, dtype: object

In [13]:
from collections import Counter
from itertools import chain

In [14]:
trial = '#hello World 20 is on Fire #fire #world'

In [15]:
def symbolcount (x,s='#'):
    return len(x.split (s)) - 1

In [16]:
def ulratio (x):
    n = sum(1 for l in x if l.isupper())
    d = sum(1 for l in x if l.islower())
    if d > 0:
        return n/d
    else:
        return 0.999

In [17]:
def nratio (x):
    n = sum(1 for l in x if l.isnumeric())
    d = len(x)
    return n/d

In [18]:
train['text'].apply (symbolcount, args=('@'))

0       0
1       0
2       0
3       0
4       0
       ..
7608    0
7609    2
7610    0
7611    0
7612    0
Name: text, Length: 7613, dtype: int64

In [19]:
def revtarget (x):
    return ~(x['target'] == 1) * 1

In [20]:
def transformer (x):
    assert isinstance(x, pd.DataFrame), 'Input (x) not a DataFrame'
    
    x['ratio'] = x['text'].apply(ulratio)
    x['nratio'] = x['text'].apply(nratio)
    x['hashcount'] = x['text'].apply(symbolcount, args=('#'))
    x['atcount'] = x['text'].apply(symbolcount, args=('@'))
    
    return x

In [21]:
train = transformer (train)
test = transformer (test)

In [22]:
r_train = train

In [23]:
r_train['target'] = revtarget (r_train)

In [24]:
train

,id,keyword,location,text,target,ratio,nratio,hashcount,atcount
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,0,0.217391,0.000000,1,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,0,0.192308,0.000000,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,0,0.018692,0.000000,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",0,0.020408,0.076923,1,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,0,0.044776,0.000000,2,0
...,...,...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,0,0.116667,0.012048,0,0
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,0,0.063158,0.000000,0,2
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,0,0.357143,0.138462,0,0
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,0,0.036364,0.000000,0,0


In [25]:
r_train.to_csv(path_or_buf='train_r.csv', index=False)
test.to_csv(path_or_buf='test.csv', index=False)
train.to_csv(path_or_buf='train.csv', index=False)

,id,keyword,location,text,target,ratio,nratio,hashcount,atcount
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,0,0.217391,0.000000,1,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,0,0.192308,0.000000,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,0,0.018692,0.000000,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",0,0.020408,0.076923,1,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,0,0.044776,0.000000,2,0
...,...,...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,0,0.116667,0.012048,0,0
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,0,0.063158,0.000000,0,2
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,0,0.357143,0.138462,0,0
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,0,0.036364,0.000000,0,0
